# Sistemas inteligentes. Aplicaciones

## Práctica 2-1. Preprocesamiento - 1

En esta práctica vas a implementar varias de las técnicas que hemos estudiado para preprocesar los textos hasta convertirlos en un conjunto de datos tabular, con el que poder trabajar posteriormente.

### Eliminación de caracteres no deseados

En esta primera parte de la práctica vas a trabajar con el fichero quijote.txt, que contiene todo el texto del libro El Quijote en texto plano. Para simplificar los siguientes ejercicios, debes quedarte solo con las palabras que aparecen, eliminando cualquier otro símbolo que aparezca (recuerda trabajar solo en minúsculas o solo en mayúsculas). A partir del texto limpio, debes construir un diccionario de frecuencias en el que las claves sean las palabras y los valores sean el número de veces que aparecen en el documento. Comprueba que las palabras del diccionario son correctas.

In [1]:
import re

frecuencias = {}

with open('quijote.txt','r',encoding='latin-1') as file:
    for linea in file:
        linea = linea.lower()
        linea = re.sub(r'[^a-záéíóúüñ\s]','',linea)
        for palabra in linea.split():
            frecuencias[palabra] = frecuencias.get(palabra,0) + 1

print(len(frecuencias))

22603


### Byte-pair encoding
Antes de *tokenizar* tus textos con los *tokenizers* ya implementados in la librería nltk, vas a realizar una implementación manual del algoritmo *byte-pair encoding*. Como hemos visto en teoría, la mayoría de los tokens que vas a obtener son las palabras como tal. Pero, si en el conjunto de test aparecen algunas palabras que no existen en tu corpus, es probable que seas capaz de detectar algunas partes de estas palabras como tokens más pequeños, que contengan información sobre su significado.

En este caso vas a dividir el conjunto de palabras que has preparado en dos diccionarios:
* el primer diccionario contiene todas aquellas palabras cuya frecuencia de aparición en El Quijote es mayor que 10
* el segundo diccionario contiene todas aquellas palabras cuya frecuencia de aparición en El Quijote es 10

Las palabras que aparecen menos de 10 veces en El Quijote las vas a ignorar. El primer conjunto que has creado, con las palabras más frecuentes, será tu corpus de entrenamiento. El segundo conjunto vas a utilizarlo para testear qué tokens es capaz de encontrar en esas palabras que no están en el corpus de entrenamiento. 

In [2]:
dic_1 = {key:value for key,value in frecuencias.items() if value >10}
dic_2 = {key:value for key,value in frecuencias.items() if value ==10}
dic_3 = {key:value for key,value in frecuencias.items() if value <10}

print(f"length dictionary 1: {len(dic_1)}")
print(f"length dictionary 2: {len(dic_2)}")
print(f"length dictionary 3: {len(dic_3)}")
print(f"Total words: {len(frecuencias)}")

length dictionary 1: 2747
length dictionary 2: 222
length dictionary 3: 19634
Total words: 22603


Una vez que tienes el corpus preparado, ya puedes crear una función para construir el vocabulario de tokens siguiendo el algoritmo byte-pair encoding. Dicha función debe recibir como parámetro de entrada el número de *tokens* nuevos (diferentes de las letras individuales) que quieres crear. 

Ejecuta esa función sobre el diccionario del corpus de entrenamiento, para obtener tu vocabulario de tokens.

In [41]:
"""
Agregar a cada palabra del corpus "_"
Funcion para obtener el vocabulario inicial 
Funcion para extraer todos los pares inciales del corpus
Funcion para identificiar el par adyacente mas frecuente y juntarlo
Actualizar el par en el texto como junto
"""

def prep_corpus(dic_1):
    corpus = {}
    for word, freq in dic_1.items():
        key_ = word + "_"
        key = " ".join(key_)
        corpus[key] = freq
    return corpus


def get_vocab(corpus):
    vocab_set = set()
    for word in corpus.keys():
        for letter in word:
            vocab_set.add(letter)     
    
    vocab_set.remove(' ')
    return list(vocab_set)


def get_pair(corpus):
    pairs = {}
    for word, freq in corpus.items():
        # h o l a : 5
        characters = word.split()
        for i in range(len(characters)-1):
            pair = (characters[i], characters[i+1])
            pairs[pair] = pairs.get(pair,0) + freq

    if pairs:
        best_pair = max(pairs, key=pairs.get)
    else:
        best_pair = None
    
    return best_pair
        

def fusion(pair, vocab, corpus):
    old_token = " ".join(pair)
    new_token = "".join(pair)
    vocab.append(new_token)

    new_corpus = {}
    for word in corpus:
        new_word = word.replace(old_token, new_token)
        #h o l a_
        new_corpus[new_word] = corpus[word]

    return new_corpus, vocab
    

def create_tokens(corpus, vocab, num_tokens):

    for i in range(num_tokens):
        pair = get_pair(corpus)
        if not pair:
            break
        corpus, vocab = fusion(pair, vocab, corpus)

    return vocab, corpus



corpus = prep_corpus(dic_1)
vocab = get_vocab(corpus)

print(f"longitud del vocabulario antes de los tokens {len(vocab)}")
vocab, corpus = create_tokens(corpus, vocab, 50)

print(f"longitud del vocabulario despues de los tokens {len(vocab)}")
print(corpus)


longitud del vocabulario antes de los tokens 32
longitud del vocabulario despues de los tokens 82
{'p r i m er a_': 92, 'p ar te_': 357, 'd el_': 2452, 'i n g en i o s o_': 16, 'h i d al g o_': 64, 'd on_': 2628, 'qui j o te_': 2156, 'de_': 17952, 'la_': 10227, 'm an ch a_': 162, 'c a p í t u lo_': 152, 'p r i m er o_': 187, 'que_': 20412, 'tr a t a_': 38, 'con di ci ó n_': 71, 'y_': 17981, 'e j er ci ci o_': 53, 'f a m o s o_': 88, 'en_': 8109, 'un_': 1910, 'l u g ar _': 343, 'cu y o_': 59, 'n o m b r e_': 175, 'no_': 6275, 'qui er o_': 257, 'h a_': 1046, 'm u ch o_': 292, 't i e m p o_': 321, 'v i v ía_': 16, 'los_': 4701, 'l an z a_': 64, 'ad ar g a_': 19, 'an t i g u a_': 17, 'r o c í n_': 16, 'f l a co_': 11, 'un a_': 1318, 'al g o_': 127, 'm á s_': 2020, 'las_': 3427, 'n o ch es_': 15, 'al g ú n_': 280, 'a ñ adi d u r a_': 14, 'tr es_': 231, 'p ar t es_': 102, 'su _': 3334, 'h a ci en d a_': 53, 'el_': 8076, 'd e l la_': 101, 's a y o_': 12, 'p ar a_': 1435, 'f i est as_': 14, 'c

Una vez implementado el algoritmo de creación del vocabulario de tokens, crea una nueva función que reciba una palabra y la devuelva dividida en tokens del vocabulario. Recuerda que para ir detectando los tokens dentro de la palabra, se buscan los tokens en el mismo orden en el que han sido creados.

In [47]:
def tokenizer_word(word, vocab_merges):
    characters = word + "_" #hola_
    tokens = " ".join(word) # h o l a _
    
    for pair in vocab_merges:
        
        old_token = " ".join(pair)
        new_token = "".join(pair)

        if old_token in tokens:
            tokens = tokens.replace(old_token, new_token)

    return tokens

word = "ingenioso_"
print(tokenizer_word(word, vocab))



i n g en i o s o_


Ejecuta tu función de tokenizar sobre las palabras del segundo diccionario que has creado, aquellas que aparecen exactamente 10 veces en El Quijote. Analiza los tokens que las forman, dependiendo del número de tokens que hayas creado en el vocabulario.

In [50]:
import random

# Probamos con 10 palabras aleatorias del diccionario 2
palabras_test = dic_2.keys()

corpus_2 = prep_corpus(dic_2)
vocab_2 = get_vocab(corpus_2)
vocab_tokens, _ = create_tokens(corpus_2, vocab_2, 50)

for p in palabras_test:
    tokens = tokenizer_word(p, vocab_tokens)
    print(f"{p:15} -> {tokens}")

viernes         -> vi er n es
poniéndose      -> p on i é n d os e
alejandro       -> al e j an d ro
pondría         -> p on d r í a
limpias         -> li m pi a s
fruto           -> f r u t o
parto           -> p ar t o
singular        -> s in gu l ar
alabado         -> al ab ad o
peregrino       -> p ere g rin o
deshacer        -> des h ac er
satisfacer      -> sa ti s f ac er
caminando       -> ca min an d o
hermosos        -> h er m os os
lenguaje        -> l en gu a j e
desaguisado     -> desa gu i sad o
profeso         -> p ro f es o
sandez          -> san de z
gordo           -> g or d o
alcaide         -> al ca i de
consentir       -> con s en ti r
andalucía       -> an d al u c í a
materia         -> m a t eri a
compás          -> co m p á s
curar           -> cu r ar
comenzaba       -> co m en z ab a
ofrece          -> o f re c e
soltó           -> s ol t ó
osaba           -> osab a
apartar         -> a p ar t ar
acertar         -> ac er t ar
yegua           -> y e gu a
ruin 

### Tokenizers de la librería NLTK

En el módulo tokenize de la librería NLTK existen varios *tokenizers* ya implementados que puedes utilizar. Destacamos:
* word_tokenize
* WordPunctTokenizer
* RegexpTokenizer
* TweetTokenizer

Analiza cómo funciona cada uno de ellos y qué resultados obtienen sobre diferentes conjuntos de textos. Ten en cuenta que mediante RegexpTokenizer puedes poner tú misma/o la expresión regular que mejor te convenga.

In [2]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk import word_tokenize, WordPunctTokenizer, RegexpTokenizer, TweetTokenizer

# Textos de ejemplo para probar los tokenizers
texto1 = "Don Quijote de la Mancha, caballero de la triste figura, luchó contra molinos."
texto2 = "¡Hola! ¿Cómo estás? Yo estoy bien... ¡Genial!"
texto3 = "Me encanta este libro!! #ElQuijote @Cervantes https://t.co/ejemplo :) <3"
texto4 = "El precio es 3.50€ y el descuento del 10%, o sea, pagas 3.15€."

textos = [texto1, texto2, texto3, texto4]

# 1. word_tokenize: tokenizador general basado en reglas (TreebankWordTokenizer + PunktSentenceTokenizer)
print("=" * 60)
print("1. word_tokenize")
print("=" * 60)
for i, texto in enumerate(textos):
    print(f"\nTexto {i+1}: {texto}")
    print(f"Tokens: {word_tokenize(texto, language='spanish')}")

# 2. WordPunctTokenizer: separa por palabras y signos de puntuación
print("\n" + "=" * 60)
print("2. WordPunctTokenizer")
print("=" * 60)
wpt = WordPunctTokenizer()
for i, texto in enumerate(textos):
    print(f"\nTexto {i+1}: {texto}")
    print(f"Tokens: {wpt.tokenize(texto)}")

# 3. RegexpTokenizer: tokeniza según una expresión regular personalizada
print("\n" + "=" * 60)
print("3. RegexpTokenizer (solo palabras: \\w+)")
print("=" * 60)
rt = RegexpTokenizer(r'\w+')  # Solo captura secuencias de caracteres alfanuméricos
for i, texto in enumerate(textos):
    print(f"\nTexto {i+1}: {texto}")
    print(f"Tokens: {rt.tokenize(texto)}")

# RegexpTokenizer con otra expresión: palabras con acentos y ñ
print("\n" + "=" * 60)
print("3b. RegexpTokenizer (palabras español: [a-záéíóúüñ]+)")
print("=" * 60)
rt2 = RegexpTokenizer(r'[a-záéíóúüñ]+')
for i, texto in enumerate(textos):
    print(f"\nTexto {i+1}: {texto.lower()}")
    print(f"Tokens: {rt2.tokenize(texto.lower())}")

# 4. TweetTokenizer: diseñado para textos de redes sociales (emoticonos, hashtags, menciones)
print("\n" + "=" * 60)
print("4. TweetTokenizer")
print("=" * 60)
tt = TweetTokenizer(preserve_case=False, strip_handles=False, reduce_len=True)
for i, texto in enumerate(textos):
    print(f"\nTexto {i+1}: {texto}")
    print(f"Tokens: {tt.tokenize(texto)}")

[nltk_data] Downloading package punkt to /Users/juancho/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/juancho/nltk_data...


1. word_tokenize

Texto 1: Don Quijote de la Mancha, caballero de la triste figura, luchó contra molinos.
Tokens: ['Don', 'Quijote', 'de', 'la', 'Mancha', ',', 'caballero', 'de', 'la', 'triste', 'figura', ',', 'luchó', 'contra', 'molinos', '.']

Texto 2: ¡Hola! ¿Cómo estás? Yo estoy bien... ¡Genial!
Tokens: ['¡Hola', '!', '¿Cómo', 'estás', '?', 'Yo', 'estoy', 'bien', '...', '¡Genial', '!']

Texto 3: Me encanta este libro!! #ElQuijote @Cervantes https://t.co/ejemplo :) <3
Tokens: ['Me', 'encanta', 'este', 'libro', '!', '!', '#', 'ElQuijote', '@', 'Cervantes', 'https', ':', '//t.co/ejemplo', ':', ')', '<', '3']

Texto 4: El precio es 3.50€ y el descuento del 10%, o sea, pagas 3.15€.
Tokens: ['El', 'precio', 'es', '3.50€', 'y', 'el', 'descuento', 'del', '10', '%', ',', 'o', 'sea', ',', 'pagas', '3.15€', '.']

2. WordPunctTokenizer

Texto 1: Don Quijote de la Mancha, caballero de la triste figura, luchó contra molinos.
Tokens: ['Don', 'Quijote', 'de', 'la', 'Mancha', ',', 'caballero', 'de'

[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
